<a href="https://colab.research.google.com/github/masies/CRA/blob/main/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html
! pip install -q transformers==3.5.0 fast-trees
! git clone -q https://github.com/microsoft/CodeXGLUE.git

from os import walk
import os
import regex

from google.colab import auth
auth.authenticate_user()
project_id = 'helical-loop-303918'
bucket_name = 'code_review_automation'
!gcloud config set project {project_id}

     |████████████████████████████████| 753.4MB 25kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
     |████████████████████████████████| 1.3MB 12.9MB/s 
     |████████████████████████████████| 2.9MB 47.8MB/s 
     |████████████████████████████████| 1.1MB 47.7MB/s 
     |████████████████████████████████| 870kB 47.4MB/s 
     |████████████████████████████████| 163kB 50.4MB/s 
     |████████████████████████████████| 112kB 51.3MB/s 
     |████████████████████████████████| 71kB 5.4MB/s 
Updated property [core/project].


In [5]:
!rm -rf ./validation_eval/
!gsutil -m cp -r gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v2/codeANDcomment_code/pretrained/validation_eval .

Copying gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v2/codeANDcomment_code/pretrained/validation_eval/codeANDcomment_code_205000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v2/codeANDcomment_code/pretrained/validation_eval/codeANDcomment_code_240000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v2/codeANDcomment_code/pretrained/validation_eval/codeANDcomment_code_210000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v2/codeANDcomment_code/pretrained/validation_eval/codeANDcomment_code_215000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v2/codeANDcomment_code/pretrained/validation_eval/codeANDcomment_code_235000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v2/codeANDcomment_code/pretrained/validation_eval/codeANDcom

In [6]:
def eval(val_folder, tasks):
  for task in tasks:
    print(task)
    _, _, filenames = next(walk(val_folder))
    prediction_paths = [os.path.join(val_folder, f) for f in filenames if f.startswith(task) and f.endswith("predictions")]
    target_path = [os.path.join(val_folder, f) for f in filenames if f.startswith(task) and f.endswith("targets")][0]

    with open(target_path) as f:
        targets = [item.strip() for item in f.readlines()]

    for path in sorted(prediction_paths):
      step = regex.findall( task + "_(\d+)_predictions", path)[0]
      if step.endswith("000"):
        # !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs ./../../../../../{target_path} --hyp ./../../../../../{path} --lang java --params 0.25,0.25,0.25,0.25
        
        with open(path) as f:
          predictions = [item.strip() for item in f.readlines()]
        
        perfect_predictions = 0
        almost_perfect_predictions = 0
        for pred, target in zip(predictions, targets):
          if ' '.join(pred.strip().split()) == ' '.join(target.strip().split()):
            perfect_predictions += 1
          elif ''.join(pred.strip().split()) == ''.join(target.strip().split()):
            almost_perfect_predictions += 1
            index = next((i for i, (char1, char2) in enumerate(zip(pred, target)) if char1 != char2), None)
            
        print(step + " - PP: ", round(perfect_predictions/len(predictions)*100,2), " | APP: ", round((perfect_predictions + almost_perfect_predictions)/len(predictions)*100,2))
        print()

# eval('validation_eval', ["code_code", "codeANDcomment_code"])
# eval('validation_eval', ["code_code"])
eval('validation_eval', ["codeANDcomment_code"])
      


codeANDcomment_code
205000 - PP:  0.0  | APP:  0.0

210000 - PP:  25.57  | APP:  26.39

215000 - PP:  26.27  | APP:  27.09

220000 - PP:  26.62  | APP:  27.44

225000 - PP:  26.62  | APP:  27.5

230000 - PP:  26.62  | APP:  27.5

235000 - PP:  27.2  | APP:  27.79

240000 - PP:  27.15  | APP:  27.85

245000 - PP:  27.03  | APP:  27.67

250000 - PP:  26.91  | APP:  27.67



In [ ]:
def codeBleu(pred, target):
  with open("pred.txt", "w") as f:
    f.write(pred)
  with open("tar.txt", "w") as f:
    f.write(target)
  try :
    a = !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs ./../../../../../pred.txt --hyp ./../../../../../tar.txt --lang java --params 0.25,0.25,0.25,0.25
    codeBleu = float(a[1].replace("CodeBLEU score:  ",""))
    print(codeBleu)
  except Exception as e:
    codeBleu = 0